# Setup

## Imports

In [278]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import re
from typing import List
from scipy.fftpack import dct
from scipy.stats import linregress
from sklearn.cluster import KMeans
from scipy.ndimage import gaussian_filter1d
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import BallTree

In [279]:
plt.rcParams["font.family"] ="NanumGothic"
plt.rcParams["axes.unicode_minus"] =False

In [280]:
# Logger config
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [281]:
train_df = pd.read_csv("./input/processed/train_df_imputed.csv")
test_df = pd.read_csv("./input/processed/test_df_imputed.csv")
station_info_processed = pd.read_csv("./input/processed/station_info_processed.csv")

In [282]:
train_df.head()

,id,station,station_name,date,cloud_cover_0,cloud_cover_1,cloud_cover_10,cloud_cover_11,cloud_cover_12,cloud_cover_13,...,wind_speed_23,wind_speed_3,wind_speed_4,wind_speed_5,wind_speed_6,wind_speed_7,wind_speed_8,wind_speed_9,climatology_temp,target
0,0,98,동두천,01-01,0.0,0.0,9.0,0.0,3.0,3.0,...,2.3,0.6,0.3,0.7,0.6,0.7,0.8,0.1,-2.707143,-3.992857
1,1,98,동두천,01-02,0.0,0.0,0.0,0.0,0.0,0.0,...,0.7,0.2,0.0,1.1,1.3,0.5,0.9,0.4,-3.646429,-1.653571
2,2,98,동두천,01-03,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,1.5,0.8,0.8,0.9,1.0,1.1,0.1,-2.694643,-0.005357
3,3,98,동두천,01-04,0.0,0.0,2.0,0.0,0.0,1.0,...,0.9,0.3,0.5,0.2,0.5,1.3,0.5,0.2,-2.501786,-0.898214
4,4,98,동두천,01-05,0.0,0.0,0.0,0.0,0.0,0.0,...,1.4,1.1,1.6,1.4,1.8,0.5,1.1,0.6,-2.625000,-1.775000


## Helper Functions

In [283]:
def extract_time_feature_bases(df: pd.DataFrame) -> List[str]:
    time_cols = [col for col in df.columns if re.match(r".+_\d{1,2}$", col)]
    return sorted({col.rsplit("_", 1)[0] for col in time_cols})


In [284]:
def hourly(df: pd.DataFrame, prefix: str) -> np.ndarray | None:
    cols = [f"{prefix}_{h}" for h in range(24)]
    valid_cols = [col for col in cols if col in df]

    if not valid_cols:
        return None

    M = df[valid_cols].astype(float).values

    per_hour_median = np.nanmedian(M, axis=0)
    M[np.isnan(M)] = np.take(per_hour_median, np.isnan(M).nonzero()[1])

    # Final global fallback (in case full column is NaN, i think they are not appearing in the dataset btw.)
    if np.isnan(M).any():
        M = np.nan_to_num(M, nan=np.nanmedian(M))

    return M

In [285]:
def check_nan(df: pd.DataFrame) -> int:
    return df.isna().sum().sum()

# Feature Engineering

In [286]:
def date_column_handler(df: pd.DataFrame) -> pd.DataFrame:
    output = df.copy()

    # Convert string to datetime(with consideration of leap years, making them all to be in 2024) for future use
    if df['date'].dtype == object and df['date'].str.match(r"^\d{2}-\d{2}$").all():
        output['date'] = pd.to_datetime('2024-' + df['date'], format='%Y-%m-%d', errors='coerce')
    else:
        raise ValueError("Check the date column!")

    # Note)
    # Model might falsely interpret the valuse as ordinal or linear(i.e., it may think Febulary is "twice as big" as Jan!)
    # So we are converting them into sin/cos
    # +) considering our target: the next day's temperature anomaly, therefore we would better to use sin/cos for day of year(doy)!
    # We use both sin/cos -> since we want to uniquely represent any position on a cycle! therefore we need 2D coords

    doy = output['date'].dt.dayofyear
    # Dataset is including the leap year -> so we use the average year length(if other better ideas, please post them at the PR!!)
    output['doy_sin'] = np.sin(2 * np.pi * doy / 365.25)
    output['doy_cos'] = np.cos(2 * np.pi * doy / 365.25)

    output = output.drop(columns=['date'])
    return output

In [287]:
def merge_with_station_data(df: pd.DataFrame, station_df: pd.DataFrame) -> pd.DataFrame:
    output = df.copy()

    output = output.merge(station_df, on='station', how='left')
    output = output.drop(columns=['station', 'station_name'])

    return output

In [288]:
def engineer_anomaly_features(df: pd.DataFrame) -> pd.DataFrame:
    output = df.copy()

    output['temp_mean'] = hourly(output, 'surface_temp').mean(axis=1)
    output['anom'] = df['temp_mean'] - df['climatology_temp']

    return output

In [289]:
def engineer_moisture_features(df: pd.DataFrame) -> pd.DataFrame:
    output = df.copy()

    temp = hourly(output, 'surface_temp')
    dew = hourly(output, 'dew_point')
    humid = hourly(output, 'humidity')

    if dew is not None:
        output['dew_gap'] = np.mean(temp - dew, axis=1)
    if humid is not None:
        output['humid_mean'] = humid.mean(axis=1)
        output['humid_range'] = humid.max(axis=1) - humid.min(axis=1)

    return output

In [290]:
def engineer_cloud_radiation(df: pd.DataFrame) -> pd.DataFrame:
    output = df.copy()

    cloud = hourly(output, 'cloud_cover')
    sun = hourly(output, 'sunshine_duration')

    if cloud is not None:
        output['cloud_mean'] = cloud.mean(axis=1)
    if sun is not None:
        sun_total = sun.sum(axis=1)
        output['sun_total'] = sun_total
        output['sun_hours'] = (sun > 0).sum(axis=1)
        output['sun_cloud_int'] = sun_total * output.get('cloud_mean', 0)

    return output

In [291]:
def engineer_pressure_features(df: pd.DataFrame) -> pd.DataFrame:
    output = df.copy()

    slp = hourly(output, 'sea_level_pressure')

    output['pressure_range'] = slp.max(axis=1) - slp.min(axis=1)
    output['pressure_trend'] = slp[:, -1] - slp[:, 0]

    return output

In [292]:
def drop_unused_features(df: pd.DataFrame) -> pd.DataFrame:
    output = df.copy()

    # Drop {prefix}_{n}(original) features
    prefixes = extract_time_feature_bases(output)
    drop_cols = [f"{p}_{h}" for p in prefixes for h in range(24)]
    existing = [col for col in drop_cols if col in df.columns]
    output = output.drop(columns=existing)

    return output

In [293]:
def engineer_features(df: pd.DataFrame, station_df: pd.DataFrame) -> pd.DataFrame:
    output = df.copy()

    # 1. date column
    output = date_column_handler(output)

    # 2. Station Geographical Features
    output = merge_with_station_data(output, station_df)

    # 3. Engineer hourly features
    output = engineer_moisture_features(output)
    output = engineer_moisture_features(output)
    output = engineer_cloud_radiation(output)
    output = engineer_pressure_features(output)

    # 4. Reduce features
    output = drop_unused_features(output)

    # 5. Check if NaN are existing
    if check_nan(output) != 0: raise ValueError("Nans are exist!")

    return output

In [294]:
train_df = engineer_features(train_df, station_info_processed)
test_df = engineer_features(test_df, station_info_processed)

In [295]:
train_df.shape

(13132, 16)

In [296]:
test_df.shape

(3004, 15)

In [297]:
train_df

,id,climatology_temp,target,doy_sin,doy_cos,elev,pressure_sensor_h,dew_gap,humid_mean,humid_range,cloud_mean,sun_total,sun_hours,sun_cloud_int,pressure_range,pressure_trend
0,0,-2.707143,-3.992857,0.017202,0.999852,115.62,116.74,11.225000,46.875000,38.0,1.791667,123.655556,10,221.549537,5.0,-1.9
1,1,-3.646429,-1.653571,0.034398,0.999408,115.62,116.74,11.704167,45.500000,52.0,0.000000,126.955556,23,0.000000,4.1,1.1
2,2,-2.694643,-0.005357,0.051584,0.998669,115.62,116.74,12.325000,43.583333,59.0,0.000000,126.977778,23,0.000000,3.6,0.4
3,3,-2.501786,-0.898214,0.068755,0.997634,115.62,116.74,8.045833,53.166667,52.0,1.208333,122.688889,11,148.249074,8.7,-8.7
4,4,-2.625000,-1.775000,0.085906,0.996303,115.62,116.74,12.337500,40.583333,66.0,0.250000,125.933333,23,31.483333,3.3,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13127,17506,-3.485714,0.785714,-0.073045,0.997329,80.09,81.59,5.245833,63.500000,50.0,0.416667,120.988889,11,50.412037,5.5,4.5
13128,17507,-2.632143,-0.367857,-0.055879,0.998438,80.09,81.59,10.525000,52.333333,39.0,0.875000,120.855556,11,105.748611,5.5,-3.9
13129,17508,-1.555357,1.055357,-0.038696,0.999251,80.09,81.59,10.404167,52.541667,43.0,0.291667,122.222222,11,35.648148,2.9,1.5
13130,17509,-2.814286,6.614286,-0.021501,0.999769,80.09,81.59,5.291667,69.625000,47.0,4.583333,119.788889,9,549.032407,4.3,-1.9


In [298]:
test_df

,id,climatology_temp,doy_sin,doy_cos,elev,pressure_sensor_h,dew_gap,humid_mean,humid_range,cloud_mean,sun_total,sun_hours,sun_cloud_int,pressure_range,pressure_trend
0,0,24.017857,-0.834370,-0.551205,39.81,40.81,4.933333,77.416667,27.0,4.416667,115.061111,8,508.186574,3.7,2.6
1,1,1.778571,-0.526755,0.850017,30.59,31.99,5.462500,70.000000,55.0,1.250000,126.200000,22,157.750000,4.6,-4.6
2,2,24.091071,-0.109446,-0.993993,30.59,31.99,5.137500,87.666667,29.0,3.125000,119.588889,13,373.715278,4.1,-2.3
3,3,27.076786,-0.691351,-0.722519,30.59,31.99,7.870833,85.708333,39.0,1.875000,123.255556,13,231.104167,2.8,-0.2
4,4,7.417857,0.969843,0.243730,30.59,31.99,11.195833,60.916667,63.0,0.000000,126.727778,23,0.000000,3.7,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,2999,1.932143,-0.467065,0.884223,39.81,40.81,10.791667,44.291667,29.0,4.750000,119.811111,8,569.102778,4.9,-3.2
3000,3000,12.312500,0.999930,-0.011826,30.59,31.99,12.650000,53.791667,62.0,1.250000,130.005556,24,162.506944,3.8,-0.3
3001,3001,27.073214,-0.391358,-0.920239,39.81,40.81,3.383333,75.833333,23.0,6.250000,111.833333,6,698.958333,3.2,0.6
3002,3002,18.208929,0.643337,-0.765584,39.81,40.81,10.691667,74.958333,53.0,5.000000,123.327778,14,616.638889,4.2,0.7


> Export

In [299]:
# export df
train_df.to_csv("./input/processed/train_df_final_v5.csv", index=False)
test_df.to_csv("./input/processed/test_df_final_v5.csv", index=False)